In [1]:
import os
import sys
sys.path.append(os.path.join("..", "..", "..", "build", "lib"))
from TbGAl.metric import EuclideanMetric
from TbGAl.multivector import *
from TbGAl.sparse_tensor import *

### Configuração do ambiente da minha biblioteca

In [2]:
# set_device(0)

In [3]:
DIMS = 10

In [4]:
Multivector.set_N(DIMS)

In [5]:
generate_T(EuclideanMetric())

In [6]:
e1 = e(1)
e2 = e(2)
e3 = e(3)
e4 = e(4)
e5 = e(5)
e6 = e(6)
e7 = e(7)
e8 = e(8)

## Funções utilitárias

In [7]:
def process_factors(factors_A, factor_B):
    '''
        Processa a lista de fatores A para um fator de B. Se o fator resultante possuir grau > 2, 
        isso significa que não é um fator e sim um versor de grau > 2, devendo ser fatorado novamente
        e adicionado à lista de fatores resultantes
    '''
    new_factors_A = []
    inv_factor_B = INVERSE(factor_B)
    new_factor = GP(factors_A[0], inv_factor_B)
    if new_factor.getGrade() == 2:
        new_factors_A += fact_versor(new_factor)[:-1]
    else:
        new_factors_A.append(new_factor)
    for i in range(1, len(factors_A)):
        new_factor = GP(GP(factor_B, factors_A[i]), inv_factor_B)
        if new_factor.getGrade() == 2:
            new_factors_A += fact_versor(new_factor)[:-1]
        else:
            new_factors_A.append(new_factor)
    return new_factors_A

In [8]:
def geometric_product_fact(factors_A, factors_B):
    '''
        Processa o conjunto de fatores de A e B. Utiliza a função process_factors como auxiliar
    '''
    new_factors = factors_A
    for f in factors_B:
        newest_factors = []
        for k in process_factors(new_factors, f):
            newest_factors.append(k)
        new_factors = newest_factors
    return new_factors

# Objetivo: Calcular AB diretamente sobre os fatores

## Sendo A = 1/2 (e1^e3^e4 - e1^e3^e5 + e2^e3^e4 - e2^e3^e5)

In [9]:
A = [0.70710678 * e1 + 0.70710678 * e2, e3, 0.70710678 * e4 - 0.70710678 * e5]
A

[0.707107e(1)+0.707107e(2), e(3), 0.707107e(4)-0.707107e(5)]

## Situação 1: B possui dependência linear com alguns fatores de A
> ### B = sqrt(2)/2 (e1^e3+e2^e3)

In [10]:
B = [0.707107 * e1 + 0.707107 * e2, e3]
B

[0.707107e(1)+0.707107e(2), e(3)]

>>### A saída do produto geométrico em sua forma não-fatorada deve ser 
>>### A B = -0.7071074376271083 * e4 + 0.7071074376271083 * e5

In [11]:
geometric_product_fact(A, B)

[1.000000e(3), -1.000000e(3), 0.707107e(4)-0.707107e(5)]

## Situação 2: B não possui dependência linear com nenhum fator de A
>### B = e6^e7^e8

In [12]:
B = [e6, e7, e8]
B

[e(6), e(7), e(8)]

>>### A saída do produto geométrico em sua forma não fatorada deve ser:
>>### A B = 1/2 (e1^e3^e4^e6^e7^e8 + e2^e3^e4^e6^e7^e8 - e1^e3^e5^e6^e7^e8 - e2^e3^e5^e6^e7^e8)

In [13]:
geometric_product_fact(A, B)

[e(8),
 -1.000000e(7),
 e(6),
 -0.707107e(1)-0.707107e(2),
 -1.000000e(3),
 -0.707107e(4)+0.707107e(5)]